<a href="https://colab.research.google.com/github/YKochura/ml_basics/blob/main/decision_tree_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Дерево рішень для класифікації у Python


Дерево рішень - це **контрольована (supervised)** модель машинного навчання, яка може використовуватися як для **класифікації**, так і для **регресії**. По суті, дерево рішень використовує деревоподібну структуру для прогнозування вихідного значення для даного вхідного прикладу. У дереві кожен шлях від кореневого вузла до листового вузла представляє *шлях прийняття рішення (decision path)*, який закінчується передбаченим значенням.

Простий приклад виглядає наступним чином:
![caption](https://github.com/YKochura/ml_basics/blob/main/figures/decision_tree.png?raw=1)

Дерева рішень мають багато переваг. Наприклад, їх легко зрозуміти, а їх рішення легко інтерпретувати. Крім того, вони не вимагають складної підготовки даних. Можна знайти більш широкий перелік їх переваг та недоліків [тут](http://scikit-learn.org/stable/modules/tree.html).

### Алгоритм навчання CART 
Для підготовки дерева рішень можна використовувати різні алгоритми. У цьому нотбуці ми зосередимось на алгоритмі *CART* (Classification and Regression Trees) для *класифікації*. Алгоритм CART створює *бінарне дерево*, в якому кожен нелистовий вузол має рівно двох дітей (що відповідає відповіді так/ні).

Враховуючи набір навчальних прикладів та їх мітки, алгоритм неодноразово розбиває навчальні приклади $D$ на дві підмножини $D_{left}, D_{right}$ використовуючи деяку ознаку $f$ та поріг ознаки $t_f$ такі, що приклади з однаковими мітками групуються разом. На кожному вузлі алгоритм вибирає розділ $\theta = (f, t_f)$, який видає найчистіші підмножини, зважені за їх розміром. Чистота вимірюється за допомогою *коефіцієнту Джині (Gini impurity, не плутати з Gini coefficient)*.

Отже, на кожному кроці алгоритм вибирає параметри $\theta$, які мінімізують наступну функцію витрат:

\begin{equation}
J(D, \theta) = \frac{n_{left}}{n_{total}} G_{left} + \frac{n_{right}}{n_{total}} G_{right}
\end{equation}

- $D$: навчальні приклади, що залишились
- $n_{total}$ : кількість навчальних прикладів, що залишились
- $\theta = (f, t_f)$: ознака та поріг ознаки
- $n_{left}/n_{right}$: кількість прикладів у лівій / правій підмножині
- $G_{left}/G_{right}$: Gini impurity лівої / правої підмножини

Цей крок повторюється рекурсивно, доки не буде досягнута *максимально допустима глибина* або поточна кількість зразків $n_{total}$ не опуститься нижче деякої мінімальної кількості. Оригінальні рівняння можна знайти [тут](http://scikit-learn.org/stable/modules/tree.html).

Після побудови дерева нові приклади можна класифікувати шляхом навігації по дереву, перевіряючи на кожному вузлі відповідну ознаку, поки не буде досягнутий листовий вузол/прогноз.


### Коефіцієнт Джині (Gini Impurity)

Враховуючи $K$ різних класифікаційних значень $k \in \{1, ..., K\}$ Gini impurity вузла $m$ бчислюється наступним чином:

\begin{equation}
G_m = 1 - \sum_{k=1}^{K} (p_{m,k})^2
\end{equation}

де $p_{m, k}$ - це частка прикладів навчання з класом $k$ серед усіх прикладів навчання у вузлі $m$.

Gini impurity можна використовувати для оцінки того, на скільки добре виконано розподіл. Розподіл ділить заданий набір навчальних прикладів на дві групи. Gini вимірює, наскільки "змішані" отримані групи. Ідеальне розділення (тобто кожна група містить лише зразки одного класу) відповідає Gini impurity = 0. Якщо отримані групи містять однаково багато зразків кожного класу, Gini impurity досягне найвищого значення 0,5.

### Застереження

Без регуляризації дерева прийняття рішень, ймовірно, виникне перенавчання. Цього можна запобігти, використовуючи такі методи, як *обрізка*, або надаючи максимально дозволену глибину дерева та/або мінімальну кількість зразків, необхідних для подальшого розділення вузла.

In [ ]:
from sklearn.datasets import load_iris
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
np.random.seed(123)

% matplotlib inline

## Набір даних

Набір даних iris поєднує в собі 150 прикладів 3 різних видів квітів ірису (Setosa, Versicolour та Virginica). Кожен приклад описується чотирма атрибутами: довжина чашолистка (см), ширина чашолистка (см), довжина пелюстки (см), ширина пелюстки (см).

In [ ]:
iris = load_iris()

X, y = iris.data, iris.target

In [ ]:
# Split the data into a training and test set
X_train, X_test, y_train, y_test = train_test_split(X, y)

print(f'Shape X_train: {X_train.shape}')
print(f'Shape y_train: {y_train.shape}')
print(f'Shape X_test: {X_test.shape}')
print(f'Shape y_test: {y_test.shape}')

Shape X_train: (112, 4)
Shape y_train: (112,)
Shape X_test: (38, 4)
Shape y_test: (38,)


## Клас дерева рішень

Деякі частини коду були взяті з [цього туторіалу](https://machinelearningmastery.com/implement-decision-tree-algorithm-scratch-python/)

In [ ]:
class DecisionTree:
    """
    Decision tree for classification
    """

    def __init__(self):
        self.root_dict = None
        self.tree_dict = None

    def split_dataset(self, X, y, feature_idx, threshold):
        """
        Splits dataset X into two subsets, according to a given feature
        and feature threshold.

        Args:
            X: 2D numpy array with data samples
            y: 1D numpy array with labels
            feature_idx: int, index of feature used for splitting the data
            threshold: float, threshold used for splitting the data

        Returns:
            splits: dict containing the left and right subsets
            and their labels
        """

        left_idx = np.where(X[:, feature_idx] < threshold)
        right_idx = np.where(X[:, feature_idx] >= threshold)

        left_subset = X[left_idx]
        y_left = y[left_idx]

        right_subset = X[right_idx]
        y_right = y[right_idx]

        splits = {
        'left': left_subset,
        'y_left': y_left,
        'right': right_subset,
        'y_right': y_right,
        }

        return splits

    def gini_impurity(self, y_left, y_right, n_left, n_right):
        """
        Computes Gini impurity of a split.

        Args:
            y_left, y_right: target values of samples in left/right subset
            n_left, n_right: number of samples in left/right subset

        Returns:
            gini_left: float, Gini impurity of left subset
            gini_right: gloat, Gini impurity of right subset
        """

        n_total = n_left + n_left

        score_left, score_right = 0, 0
        gini_left, gini_right = 0, 0

        if n_left != 0:
            for c in range(self.n_classes):
                # For each class c, compute fraction of samples with class c
                p_left = len(np.where(y_left == c)[0]) / n_left
                score_left += p_left * p_left
            gini_left = 1 - score_left

        if n_right != 0:
            for c in range(self.n_classes):
                p_right = len(np.where(y_right == c)[0]) / n_right
                score_right += p_right * p_right
            gini_right = 1 - score_right

        return gini_left, gini_right

    def get_cost(self, splits):
        """
        Computes cost of a split given the Gini impurity of
        the left and right subset and the sizes of the subsets
        
        Args:
            splits: dict, containing params of current split
        """
        y_left = splits['y_left']
        y_right = splits['y_right']

        n_left = len(y_left)
        n_right = len(y_right)
        n_total = n_left + n_right

        gini_left, gini_right = self.gini_impurity(y_left, y_right, n_left, n_right)
        cost = (n_left / n_total) * gini_left + (n_right / n_total) * gini_right

        return cost

    def find_best_split(self, X, y):
        """
        Finds the best feature and feature index to split dataset X into
        two groups. Checks every value of every attribute as a candidate
        split.

        Args:
            X: 2D numpy array with data samples
            y: 1D numpy array with labels

        Returns:
            best_split_params: dict, containing parameters of the best split
        """

        n_samples, n_features = X.shape

        best_feature_idx, best_threshold, best_cost, best_splits = np.inf, np.inf, np.inf, None

        for feature_idx in range(n_features):
            for i in range(n_samples):
                current_sample = X[i]
                threshold = current_sample[feature_idx]
                splits = self.split_dataset(X, y, feature_idx, threshold)
                cost = self.get_cost(splits)

                if cost < best_cost:
                    best_feature_idx = feature_idx
                    best_threshold = threshold
                    best_cost = cost
                    best_splits = splits

        best_split_params = {
            'feature_idx': best_feature_idx,
            'threshold': best_threshold,
            'cost': best_cost,
            'left': best_splits['left'],
            'y_left': best_splits['y_left'],
            'right': best_splits['right'],
            'y_right': best_splits['y_right'],
        }

        return best_split_params


    def build_tree(self, node_dict, depth, max_depth, min_samples):
        """
        Builds the decision tree in a recursive fashion.

        Args:
            node_dict: dict, representing the current node
            depth: int, depth of current node in the tree
            max_depth: int, maximum allowed tree depth
            min_samples: int, minimum number of samples needed to split a node further

        Returns:
            node_dict: dict, representing the full subtree originating from current node
        """
        left_samples = node_dict['left']
        right_samples = node_dict['right']
        y_left_samples = node_dict['y_left']
        y_right_samples = node_dict['y_right']

        if len(y_left_samples) == 0 or len(y_right_samples) == 0:
            node_dict["left_child"] = node_dict["right_child"] = self.create_terminal_node(np.append(y_left_samples, y_right_samples))
            return None

        if depth >= max_depth:
            node_dict["left_child"] = self.create_terminal_node(y_left_samples)
            node_dict["right_child"] = self.create_terminal_node(y_right_samples)
            return None

        if len(right_samples) < min_samples:
            node_dict["right_child"] = self.create_terminal_node(y_right_samples)
        else:
            node_dict["right_child"] = self.find_best_split(right_samples, y_right_samples)
            self.build_tree(node_dict["right_child"], depth+1, max_depth, min_samples)

        if len(left_samples) < min_samples:
            node_dict["left_child"] = self.create_terminal_node(y_left_samples)
        else:
            node_dict["left_child"] = self.find_best_split(left_samples, y_left_samples)
            self.build_tree(node_dict["left_child"], depth+1, max_depth, min_samples)

        return node_dict

    def create_terminal_node(self, y):
        """
        Creates a terminal node.
        Given a set of labels the most common label is computed and
        set as the classification value of the node.

        Args:
            y: 1D numpy array with labels
        Returns:
            classification: int, predicted class
        """
        classification = max(set(y), key=list(y).count)
        return classification

    def train(self, X, y, max_depth, min_samples):
        """
        Fits decision tree on a given dataset.

        Args:
            X: 2D numpy array with data samples
            y: 1D numpy array with labels
            max_depth: int, maximum allowed tree depth
            min_samples: int, minimum number of samples needed to split a node further
        """
        self.n_classes = len(set(y))
        self.root_dict = self.find_best_split(X, y)
        self.tree_dict = self.build_tree(self.root_dict, 1, max_depth, min_samples)

    def predict(self, X, node):
        """
        Predicts the class for a given input example X.

        Args:
            X: 1D numpy array, input example
            node: dict, representing trained decision tree

        Returns:
            prediction: int, predicted class
        """
        feature_idx = node['feature_idx']
        threshold = node['threshold']

        if X[feature_idx] < threshold:
            if isinstance(node['left_child'], (int, np.integer)):
                return node['left_child']
            else:
                prediction = self.predict(X, node['left_child'])
        elif X[feature_idx] >= threshold:
            if isinstance(node['right_child'], (int, np.integer)):
                return node['right_child']
            else:
                prediction = self.predict(X, node['right_child'])

        return prediction

## Ініціалізація та навчання дерева рішень

In [ ]:
tree = DecisionTree()
tree.train(X_train, y_train, max_depth=2, min_samples=1)

## Вивід структури дерева рішень

In [ ]:
def print_tree(node, depth=0):
    if isinstance(node, (int, np.integer)):
        print(f"{depth * '  '}predicted class: {iris.target_names[node]}")
    else:
        print(f"{depth * '  '}{iris.feature_names[node['feature_idx']]} < {node['threshold']}, "
             f"cost of split: {round(node['cost'], 3)}")
        print_tree(node["left_child"], depth+1)
        print_tree(node["right_child"], depth+1)

In [ ]:
print_tree(tree.tree_dict)

petal length (cm) < 3.0, cost of split: 0.346
  sepal length (cm) < 5.4, cost of split: 0.0
    predicted class: setosa
    predicted class: setosa
  petal width (cm) < 1.8, cost of split: 0.097
    predicted class: versicolor
    predicted class: virginica


## Тестування дерева рішень

In [ ]:
all_predictions = []
for i in range(X_test.shape[0]):
    result = tree.predict(X_test[i], tree.tree_dict)
    all_predictions.append(y_test[i] == result)

print(f"Accuracy on test set: {sum(all_predictions) / len(all_predictions)}")

Accuracy on test set: 0.9473684210526315


## Переваги та недоліки

### Переваги:
 - Простота, зручність візуалізації.
 - Нескладна підготовка даних, не виманає нормалізації, однак модель не підтримує відсутні значення.
 - Вартість прогнозування є логарифмічною функцією від кількості точок даних;
 - Здатність одробляти як числові так і категоріальні типи даних. Інші методи, як правило, спеціалізуються на аналізі наборів даних, які мають лише один тип змінних.
 - Здатність вирішувати проблеми з кількома виходами.
 - Використовується модель білої коробки. (Результат моделі є відносно простим для інтерпретації.
 - Можна перевірити модель за допомогою статистичних тестів. Це дає змогу враховувати надійність моделі.
 
### Недоліки
 - Перенавчання.
 - Дерева рішень можуть бути нестабільними, оскільки невеликі зміни даних можуть призвести до генерування зовсім іншого дерева. Цю проблему пом’якшує використання дерев рішень у межах ансамблю.
 - Відомо, що проблема вивчення дерева оптимальних рішень є NP-повною за кількох аспектів оптимальності і навіть для простих концепцій. Отже, практичні алгоритми навчання на дереві рішень засновані на евристичних алгоритмах, таких як жадібний алгоритм, де локально оптимальні рішення приймаються на кожному вузлі. Такі алгоритми не можуть гарантувати повернення глобально оптимального дерева рішень. Це можна пом'якшити шляхом навчання кількох дерев в ансамблі, який навчається, де функції та вибірки вибірково вибираються із заміною.
 - Є поняття, які важко вивчити, оскільки дерева рішень не виражають їх легко, наприклад, XOR, проблеми паритету чи мультиплексора.

# Джерело
[Decision tree for classification](https://github.com/zotroneneis/machine_learning_basics/blob/master/decision_tree_classification.ipynb)

Переклад: Осіпов Іван, Бровченко Анастасія студенти ІО-01мп, 2020 рік